In [53]:
import requests
import pandas as pd
from io import BytesIO
import zipfile
import io
import os
import datetime
from pathlib import Path
import numpy as np
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime
import timeit
from copy import deepcopy


In [70]:
symbol = '"NIFTY"'
expiry = "'27-Aug-2020'"
engine = create_engine('sqlite:///C:\\Users\\Tathagat\\Desktop\\BhavCopy\\SqLite\\foo.db' , echo=False)
backPeriod = 5
multiplier = 5

In [55]:
query = "SELECT * from users WHERE SYMBOL=" + symbol + "and EXPIRY_DT=" + expiry + "and (INSTRUMENT='OPTIDX' or INSTRUMENT='OPTSTK') "
temp = engine.execute(query).fetchall()
temp = pd.DataFrame(temp)
columnNames = engine.execute(" PRAGMA table_info(users) ")
columnNames = pd.DataFrame(columnNames)
columnNames = columnNames[1]
temp = temp.rename(columns=columnNames)
temp = temp.drop(temp.columns[[0,-1]] , axis=1)


In [56]:
temp['TIMESTAMP'] = temp['TIMESTAMP'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y"))
temp['EXPIRY_DT'] = temp['EXPIRY_DT'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y"))
temp = temp.sort_values('TIMESTAMP')
z = {}

In [57]:
start = timeit.default_timer()

for i in temp.index :
    x = temp.loc[i]
    expiry = x['EXPIRY_DT']
    strike = x['STRIKE_PR']
    option = x['OPTION_TYP']
    key = (expiry,strike,option)
    if key in z.keys() :
        z[key].append(x)
    else :
        z[key] = []
        z[key].append(x)
        
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  5.226119299999937


In [90]:
finalResult = []
for key in z.keys() :
    x = z[key]
    x = pd.DataFrame(x)
    x['average'] = 0
    index = x.index
    total = 0
    for idx in range(len(index)) :
        if idx>=backPeriod :
            average = total/backPeriod
            if x.loc[index[idx] , 'CONTRACTS'] >= multiplier*average  :
                x.loc[index[idx] , 'average'] = average
                finalResult.append(x.loc[index[idx]])
            total = total + x.loc[index[idx] , 'CONTRACTS']
            total = total - x.loc[index[idx - backPeriod] , 'CONTRACTS']
        else :
            total = total + x.loc[index[idx] , 'CONTRACTS']
        
    
    

In [86]:
finalResult = pd.DataFrame(finalResult)

In [89]:
finalResult[(finalResult['CONTRACTS']!=0) & (finalResult['average']>10)]

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,average
5974,OPTIDX,NIFTY,2020-08-27,8000.0,CE,3130.00,3151.00,3040.00,3040.50,3166.10,234,1943.35,21300,16800,2020-07-22,14.2
7114,OPTIDX,NIFTY,2020-08-27,8000.0,CE,3233.45,3267.00,3050.00,3065.70,3065.70,1854,15525.53,219225,135975,2020-07-30,195.8
7237,OPTIDX,NIFTY,2020-08-27,9300.0,PE,7.20,15.00,6.00,8.90,8.90,511,3567.29,20925,7575,2020-07-30,64.6
7235,OPTIDX,NIFTY,2020-08-27,9200.0,PE,7.40,8.00,5.10,5.90,5.90,829,5723.79,45600,22875,2020-07-30,129.8
7427,OPTIDX,NIFTY,2020-08-27,9100.0,PE,5.10,9.75,5.10,5.85,5.85,260,1775.78,14550,5700,2020-07-31,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7789,OPTIDX,NIFTY,2020-08-27,12650.0,CE,8.40,14.00,4.15,4.75,4.75,328,3113.09,2925,1050,2020-08-04,19.4
10913,OPTIDX,NIFTY,2020-08-27,12550.0,CE,0.60,0.70,0.10,0.20,0.20,353,3322.74,16350,14475,2020-08-26,17.0
7402,OPTIDX,NIFTY,2020-08-27,12700.0,CE,5.50,5.95,3.60,5.15,5.15,3302,31462.30,55050,28125,2020-07-31,250.8
8373,OPTIDX,NIFTY,2020-08-27,12750.0,CE,3.95,6.95,2.10,3.10,3.10,132,1262.52,2700,1350,2020-08-07,26.4
